In [1]:
# coding: utf-8
import params as p
import pandas as pd
import datetime
from openpyxl import Workbook # Workbook is a class object; a container for all other parts of the document
#import os # Operating System dependent module
import pandas as pd
#import numpy as np
import re
#import pprint
from string import punctuation
#from sklearn.feature_extraction.text import CountVectorizer
#from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
#from nltk.stem.porter import PorterStemmer
#from gensim import corpora, models
#mport gensim
import pandas as pd
import re
#from sklearn.preprocessing import MinMaxScaler
from nltk.corpus import stopwords
#import glob # glob module finds all the pathnames matching a specified pattern
#from collections import Counter
import rake
#import operator
import data_aggregate as da
import Rake_TF_IDF_Cleansing as bc
import data_collection as dc
#import pandas as pd
#import numpy as np
import teradata
import getpass
from teradata import DatabaseError
print('libraries imported')

libraries imported


In [3]:
# In[3]:
t1 = datetime.datetime.now()
#Point path to an excel file with chat transcripts
combinedData_all = pd.read_excel("/Users/mdobbali/Google Drive/Overstock/Projects/cs_text_classification/Capstone_Demo/Data_Files/input_file.xlsx")
print('import done')
t2 = datetime.datetime.now()
print(t2 - t1)

import done
0:00:02.997181


In [5]:

#Comment one of the following
#output = 'combined' # if you want keywords related to all issues ;
#output = 'ind' # if you want individual issue keywords

issue_name = 'issue'
custoragent = 'customer'
#tf_idf_or_rake = 'rake' #No or Yes

#min_word_phrase_len = 4
#min_word_len = 4
#freq = 6
# In[6]:

combinedData_sample = combinedData_all.sample(frac= p.frac)
combinedData_sample = combinedData_sample.rename(index = str, columns={'incdnt_note_desc':'Chat','issue_name':'File_Name'})
if p.output == 'combined':
    combinedData_sample['File_Name'] = 'issue'

combinedData_sample.drop(combinedData_sample.columns[[1]], axis=1, inplace=True)



In [7]:
combinedData_sample = combinedData_sample.head(1)

In [9]:
message = 'basic cleaning on data is done'
t1 = datetime.datetime.now()
clean_df_rake = bc.basic_cleaning(combinedData_sample,'Chat',p.tf_idf_or_rake )
t2 = datetime.datetime.now()
print(t2 - t1)

names and time stamps have not been removed
0:00:00.039187


/Users/mdobbali/Google Drive/Overstock/Projects/cs_text_classification/Capstone_Demo/Rake_TF_IDF_Cleansing.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  chat_transcript_df[column_name] = [chat.lower() for chat in chat_transcript_df[column_name]]


In [13]:
combinedData_sample['Chat'][0]

'full transcript (includes private messages)<br /> --------------------------------------------<br /> [10:29:40 pm] hi, my name is dustin r.. how may i help you?<br /> [10:29:56 pm] visitor: hello dustin.<br /> [10:30:13 pm] visitor: could you tell me the status of my purchase?<br /> [10:30:15 pm] dustin r.: hi there!<br /> [10:30:23 pm] dustin r.: could you confirm your name and billing address for security purposes so i can get the green light to investigate?<br /> [10:30:53 pm] visitor: carmen leticia rosas<br /> [10:31:08 pm] visitor: 121 orange ave spc 32 chula vista ca. 91911<br /> [10:31:27 pm] dustin r.: thank you, is this for the &nbsp;<br /> traveler&#39;s club euro value ii collection deluxe 4-piece travel set<br /> ?<br /> [10:31:49 pm] visitor: yes it is<br /> [10:31:57 pm] dustin r.: alright, it looks like we&#39;re still investigating the tracking on the item. &nbsp;this can take 1-2 business days. &nbsp;today is the first business day.<br /> [10:32:52 pm] visitor: the p

In [12]:
clean_df_rake['Chat'][0]

'full transcript (includes private messages) -------------------------------------------- [10:29:40 pm] hi, my name is dustin r.. how may i help you? [10:29:56 pm] visitor: hello dustin. [10:30:13 pm] visitor: could you tell me the status of my purchase? [10:30:15 pm] dustin r.: hi there! [10:30:23 pm] dustin r.: could you confirm your name and billing address for security purposes so i can get the green light to investigate? [10:30:53 pm] visitor: carmen leticia rosas [10:31:08 pm] visitor: 121 orange ave spc 32 chula vista ca. 91911 [10:31:27 pm] dustin r.: thank you, is this for the  travelers club euro value ii collection deluxe 4-piece travel set ? [10:31:49 pm] visitor: yes it is [10:31:57 pm] dustin r.: alright, it looks like were still investigating the tracking on the item. this can take 1-2 business days. today is the first business day. [10:32:52 pm] visitor: the purchase is since 12/03/2015, and i need it for travel, [10:33:37 pm] visitor: 1-2 busines days is too long time 

In [15]:
message = 'extracting customer chat in a list '
t1 = datetime.datetime.now()
clean_df_rake['cust_chat'] = da.get_chat_df(clean_df_rake,'Chat',p.custoragent)
t2 = datetime.datetime.now()

In [17]:
clean_df_rake['cust_chat'][0]

[' hello dustin. ',
 ' could you tell me the status of my purchase? ',
 ' carmen leticia rosas ',
 ' 121 orange ave spc 32 chula vista ca. 91911 ',
 ' yes it is ',
 ' the purchase is since 12/03/2015, and i need it for travel, ',
 ' 1-2 busines days is too long time for the info i need ',
 ' money has collect ',
 ' ok will wait at tomorrow half day, i need and i believe in the company ']

In [19]:
message = 'getting rid of address'
t1 = datetime.datetime.now()
clean_df_rake['cust_chat'] = [bc.strip_address(lines,'/Users/mdobbali/Google Drive/Overstock/Projects/cs_text_classification/Final_Code/Data_Files/state_names.txt') for lines in clean_df_rake['cust_chat']]
t2  = datetime.datetime.now()

In [20]:
clean_df_rake['cust_chat'][0]

[' hello dustin. ',
 ' could you tell me the status of my purchase? ',
 ' carmen leticia rosas ',
 ' 121 orange ave spc 32 chula vista ca. 91911 ',
 ' yes it is ',
 ' the purchase is since 12/03/2015, and i need it for travel, ',
 ' 1-2 busines days is too long time for the info i need ',
 ' money has collect ',
 ' ok will wait at tomorrow half day, i need and i believe in the company ']

In [22]:
message = 'getting rid of one word sentences'
t1 = datetime.datetime.now()
clean_df_rake['cust_chat'] = [bc.get_one_word(lines) for lines in clean_df_rake['cust_chat']]

In [23]:
clean_df_rake['cust_chat'][0]

['hello dustin.',
 'could you tell me the status of my purchase?',
 'carmen leticia rosas',
 '121 orange ave spc 32 chula vista ca. 91911',
 'yes it is',
 'the purchase is since 12/03/2015, and i need it for travel,',
 '1-2 busines days is too long time for the info i need',
 'money has collect',
 'ok will wait at tomorrow half day, i need and i believe in the company']

In [24]:
message = 'stiriping numbers'
t1 = datetime.datetime.now()
clean_df_rake['cust_chat'] = [bc.strip_numbers_list(lines) for lines in clean_df_rake['cust_chat']]

In [25]:
clean_df_rake['cust_chat'][0]

['hello dustin.',
 'could you tell me the status of my purchase?',
 'carmen leticia rosas',
 ' orange ave spc  chula vista ca. ',
 'yes it is',
 'the purchase is since //, and i need it for travel,',
 '- busines days is too long time for the info i need',
 'money has collect',
 'ok will wait at tomorrow half day, i need and i believe in the company']

In [27]:
message ='customer chat in a paragraphs'
t1 = datetime.datetime.now()
clean_df_rake['cust_chat'] = [da.combine_list_of_items(chat,'') for chat in clean_df_rake['cust_chat']]

In [28]:
clean_df_rake['cust_chat'][0]

'hello dustin.could you tell me the status of my purchase?carmen leticia rosas orange ave spc  chula vista ca. yes it isthe purchase is since //, and i need it for travel,- busines days is too long time for the info i needmoney has collectok will wait at tomorrow half day, i need and i believe in the company'

In [ ]:

t1 = datetime.datetime.now()
dnew ={}
for item in set(clean_df_rake['File_Name']):
    list_of_chat = list(clean_df_rake[clean_df_rake['File_Name'] == item]['cust_chat'])
    dnew["{0}".format(item)]= da.combine_list_of_items(list_of_chat,'')
t2 = datetime.datetime.now()
print(t2 - t1)
p.slack_notify( str(t2 - t1) + ' ' + message)

df_issues = pd.DataFrame.from_dict(dnew, orient='index', dtype=None)

df_issues = df_issues.rename(index=str, columns={ 0 : "Chat_Para_Customer"})

if p.output == 'ind':
    issue_list_names = list(set(clean_df_rake['File_Name']))
else:
    issue_list_names = ['issue']
print(len(issue_list_names))
#issue_list_names = list(set(clean_df_rake['File_Name']) - set(['mastercard','coupon','store_card','orders']))

rake_object = rake.Rake(p.path + "SmartStoplist.txt",p.min_word_len, p.min_word_phrase_len, p.freq)

writer = pd.ExcelWriter('Data_Files/Keyword_Extracted.xlsx')
for item in issue_list_names:
    t1 = datetime.datetime.now()
    print(item)
    message = 'rake started'
    p.slack.chat.post_message('@manoj', 'rake process started for ' + str(item) + ' at ' + str(t1))
    chat_para = df_issues.loc[[item]]['Chat_Para_Customer'][0]
    keywords_tuples = rake_object.run(chat_para)
    df = pd.DataFrame(keywords_tuples)
    df.to_excel(writer,str(item))
    #df.to_excel( str(item) + '.xlsx')
    t2 = datetime.datetime.now()
    message = 'rake is done'
    p.slack_notify( str(t2 - t1) + ' ' + message)
writer.save()

#writer = pd.ExcelWriter('Keyword_Extracted.xlsx')
#for chat_issue in clean_df_rake['File_Name']:
#    temp_df.to_excel(writer,'sheet name')
